In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
# https://overface.tistory.com/573
# imshow 처리 방법
from sklearn.cluster import KMeans
!pip install utils
# 그냥 import utils 하면 안됨
import matplotlib.pyplot as plt
import argparse
import cv2
import urllib.request

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pd.set_option('mode.chained_assignment',  None) # 경고 off

In [ ]:
# 크롤링한 데이터의 url 을 통해 썸네일 이미지를 가져오기
# 이 함수를 간단히 설명하자면 request를 통해 url의 있는 이미지 정보를 가져오고
# 그 url을 읽어서 bytearray로 감싸주고 그걸 다시 uint8 형식의 np.array로 변환한다.
# 그리고 그것을 디코딩하여 3차원 이미지 np.array형식으로 바꿔 리턴
# 어렵게 설명했지만 함수이름대로 그냥 이미지 주소를 학습하기 좋은 3차원 행렬 데이터로 바꿔준다고 보면된다.

def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype='uint8')
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)

  return image

In [ ]:
# 음악 카테고리에 대한 크롤링 정보 가져와서 dataframe으로 저장
df_music=pd.read_excel('/content/drive/My Drive/음악카테고리.xlsx', index_col = 0)
pd.set_option('display.max_colwidth',1)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 30)
df_music = pd.DataFrame(df_music)

In [ ]:
# 동물 카테고리에 대한 크롤링 정보 가져와서 dataframe으로 저장
df_animal=pd.read_excel('/content/drive/My Drive/동물카테고리.xlsx', index_col = 0)
pd.set_option('display.max_colwidth',1)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 30)
df_animal = pd.DataFrame(df_animal)

In [ ]:
# 음악 카테고리 dataframe에 H,S,V 열 추가
df_music["H"]=""
df_music["S"]=""
df_music["V"]=""

In [ ]:
# 동물 카테고리 dataframe에 H,S,V 열 추가
df_animal["H"]=""
df_animal["S"]=""
df_animal["V"]=""

In [ ]:
df_animal

,channel_id,channel_title,video_id,title,thumbnail,description,publishedAt,subscriber_count,views,view_per_day,H,S,V
0,UCT60XBtfRQzf5NKFGDNbfCw,National Geographic Wild France,4__iv9Bk4bE,Le crocodile du Nil a-t-il été tué par l&#39;un de ces requins ?,https://i.ytimg.com/vi/4__iv9Bk4bE/mqdefault.jpg,"Dans une zone où cohabitent requins-tigres, requins-bouledogues, requins cuivres et requins blancs, une tête de crocodile du Nil ...",2022-06-01T17:15:00Z,1410000,56571,4040.79,,,
1,UCJX3raztRR6SAMIauK8qmBg,Alen AxP,vvVcn-9hj7c,Blue Budgie Pipi #shorts,https://i.ytimg.com/vi/vvVcn-9hj7c/mqdefault.jpg,Watch my video here https://youtu.be/WbZp5963Qm4 SUBSCRIBE HERE: https://bit.ly/1MhbNy8 DISCORD: https://bit.ly/34qcg8e ...,2022-06-01T13:06:35Z,711000,80747,5767.64,,,
2,UCAoj1O_pb7N12tFtzwGu21w,KBS 동물의 왕국,3srBkXqqbwg,[FULL VERSION] 온갖 술수가 난무하는 동물들의 대환장 생존경쟁😱ㅣ기발한 생존경쟁 1편 # Off the Fence #KBS #동물의왕국,https://i.ytimg.com/vi/3srBkXqqbwg/mqdefault.jpg,사자 #사냥 #생존 ▷ 제목: 기발한 생존경쟁 ▷ 원제: Wildest Survival – The Hunt ▷ 제작사 – Off the Fence Productions(네덜란드) ...,2022-06-01T07:00:01Z,79500,46795,3342.50,,,
3,UCT60XBtfRQzf5NKFGDNbfCw,National Geographic Wild France,9iXSnic5c5c,Violent combat entre plusieurs clans de lions,https://i.ytimg.com/vi/9iXSnic5c5c/mqdefault.jpg,"Alors que ce clan de lions chasse des zèbres, plusieurs lions inconnus surgissent, attirés par le festin. Un violent combat éclate ...",2022-05-31T17:15:00Z,1410000,26837,1789.13,,,
4,UCgsgpELbaTiGjCHIjHVF8uQ,TV생물도감,9gj2sc9qeWo,진짜 역대급입니다 발목깊이의 개울에 최고급어종 쏘가리가 바글바글 모여있네요ㄷㄷ,https://i.ytimg.com/vi/9gj2sc9qeWo/mqdefault.jpg,안녕하세요 생물도감입니다. 지난 4월 30일 쏘가리 금어기가 시작되기 전에 쏘가리 얼굴을 보고자 채나아빠님과 쏘가리 족대질 ...,2022-05-31T09:30:02Z,561000,86757,5783.80,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6649,UCNoNEF64ijgxK6OSTwdHOFA,Girl Talks Fish,8O8C5cCE_eI,4 EASY Tank Setups for a 20-Gallon Aquarium,https://i.ytimg.com/vi/8O8C5cCE_eI/mqdefault.jpg,"Here are 4 of my creative 20-gallon aquarium ideas using community fish, axolotls, and live plants. SUBSCRIBE so you won't ...",2019-06-01T13:00:03Z,170000,420520,378.85,,,
6650,UCWaYP51K6ayGFNZ9j5TdjoA,Míša Růžičková,jAF17uBgjvc,Míša Růžičková - Pan Bůůů (Minidisko Cvičíme s Míšou),https://i.ytimg.com/vi/jAF17uBgjvc/mqdefault.jpg,Ke dni dětí jsme pro vás připravili nový videoklip Pan Bůůů Máte rádi jogurty? Já moc. Co si dáme na svačinku? Jogurt nebo ...,2019-06-01T08:12:09Z,321000,1383428,1246.33,,,
6651,UCfShWs-d9Kx8zmiKULvog2g,De Buitendienst,1iUx11fWvaE,Kan een ijsbeer zich vervelen in de dierentuin? | De Buitendienst over Wilde dieren,https://i.ytimg.com/vi/1iUx11fWvaE/mqdefault.jpg,"Ijsberen in de dierentuin zijn leuk om naar te kijken, maar is het eigenlijk wel normaal om wilde dieren achter tralies te zetten?",2019-06-01T05:00:00Z,28600,228217,205.60,,,
6652,UCiFYUP4_TI70yCkkVJAlxoA,EBS 컬렉션 - 사이언스,MAz39BW8AvQ,피해 주민이 직접 찍은 생생한 산불 영상,https://i.ytimg.com/vi/MAz39BW8AvQ/mqdefault.jpg,"지구가_보내는_경고 #인류가_만든_재앙 #인류세 2016년, 캘리포니아를 삼킨 재앙적 대산불. 순식간에 여의도의 면적 11배를 태워 ...",2019-06-01T03:00:03Z,552000,776,0.70,,,


In [ ]:
len(df_music)

4919

In [ ]:
len(df_animal)

6654

In [ ]:
rgb_pixel_list_music=[]
image_pixel_list_music=[]
rgb_pixel_list_animal=[]
image_pixel_list_animal=[]

In [ ]:
clt = KMeans(n_clusters =1)

In [ ]:
##HSV##

for i in range(0,len(df_animal)):
   try:
     # thumbnail 열에 있는 url 통해 이미지로 가져오기 (위에서 정의한 url_to_image 사용)
    image = url_to_image(df_animal['thumbnail'][i])
     # BGR을 HSV로 가져오기
    image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

    # 가져온 image의 pixel 값을 저장
    image_pixel_list_animal.append(image)
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    #위에서 만든 모델에 image fitting
    clt.fit(image)
    rgb_pixel_list_animal.append(clt.cluster_centers_)
    df_animal["H"][i]=round(clt.cluster_centers_[0][0],2)*2
    df_animal["S"][i]=round(clt.cluster_centers_[0][1],2)
    df_animal["V"][i]=round(clt.cluster_centers_[0][2],2)

   except Exception:
    pass

In [ ]:
##HSV##
for i in range(0,len(df_music)):
   try:
    # thumbnail 열에 있는 url 통해 이미지로 가져오기 (위에서 정의한 url_to_image 사용)
    image = url_to_image(df_music['thumbnail'][i])
    image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    
    # 가져온 image의 pixel 값을 저장
    image_pixel_list_music.append(image)
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    #위에서 만든 모델에 image fitting
    clt.fit(image)
    rgb_pixel_list_music.append(clt.cluster_centers_)
    df_music["H"][i]=round(clt.cluster_centers_[0][0],2)*2
    df_music["S"][i]=round(clt.cluster_centers_[0][1],2)
    df_music["V"][i]=round(clt.cluster_centers_[0][2],2)

   except Exception:
    pass

In [ ]:
#BGR을 RGB 값으로 가져오기
for i in range(0,len(df_music)):
   try:
    # thumbnail 열에 있는 url 통해 이미지로 가져오기 (위에서 정의한 url_to_image 사용)
    image = url_to_image(df_music['thumbnail'][i])
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    
    # 가져온 image의 pixel 값을 저장
    image_pixel_list_music.append(image)
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    #위에서 만든 모델에 image fitting
    clt.fit(image)
    rgb_pixel_list_music.append(clt.cluster_centers_)
    df_music["R"][i]=clt.cluster_centers_[0][0]
    df_music["G"][i]=clt.cluster_centers_[0][1]
    df_music["B"][i]=clt.cluster_centers_[0][2]

   except Exception:
    pass

In [ ]:
df_music

,channel_id,channel_title,video_id,title,thumbnail,description,publishedAt,subscriber_count,views,view_per_day,H,S,V
0,UCNwQIsDSb-3y5Bhx52rATnw,lesliegraceVEVO,Kd2p_dEG-5U,"Leslie Grace, Boza - Como la Primera Vez (Official Video)",https://i.ytimg.com/vi/Kd2p_dEG-5U/mqdefault.jpg,"Leslie Grace, Boza - Como la Primera Vez (Official Video) Música: Apple Music: ...",2022-05-31T22:00:14Z,490000,587688,45206.77,67.98,74.1,40.81
1,UC5e_2fVZJ2heCb799A04AtA,TrainVEVO,uWW1V_-jC6k,"Train, Sofía Reyes - Cleopatra (Official Video)",https://i.ytimg.com/vi/uWW1V_-jC6k/mqdefault.jpg,"""Cleopatra"" ft. Sofia Reyes is out NOW: https://train.lnk.to/cleopatra Our new album 'AM Gold' - out everywhere now: ...",2022-05-31T19:00:30Z,900000,242412,18647.08,33.04,114.01,127.18
2,UC_mceLIVkQun_Xcu1KBmfdw,GMMTV RECORDS,QOiRhDTCPc0,Angel Baby Covered by Joong Archen,https://i.ytimg.com/vi/QOiRhDTCPc0/mqdefault.jpg,Angel Baby Covered by Joong Archen [Cover songs by GMMTV artists] ฟังเพลง “Angel Baby” ได้แล้ววันนี้ ...,2022-05-30T07:00:12Z,3200000,604292,43163.71,44.24,88.19,165.37
3,UCwZMmIVr07VLAfNcuqg9LZA,다비치 DAVICHI,pTk0-LKwbuU,이해리가 가득한 강민경 폰 사진 털기,https://i.ytimg.com/vi/pTk0-LKwbuU/mqdefault.jpg,이해리도 몰랐던 강민경 사진첩의 비밀 #다비치 #DAVICHI #Season_Note #이해리 #LEEHAERI #강민경 #KANGMINKYUNG.,2022-05-29T13:00:22Z,219000,494939,32995.93,113.48,77.01,92.82
4,UC3Df9-SvQY9BbfCmyvR0J5A,SebastianYatraVEVO,sXhJWCJE-5k,Sebastián Yatra - Tacones Rojos (Jimmy Kimmel Live!),https://i.ytimg.com/vi/sXhJWCJE-5k/mqdefault.jpg,Music video by Sebastián Yatra performing Tacones Rojos (Jimmy Kimmel Live!). Universal Music Latino; © 2022 UMG ...,2022-05-27T22:51:08Z,2810000,541506,31853.29,90.82,231.0,70.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,UC-ciXrv4h_IE3ga1dMfTUdA,scroll忍,87aJyyoAREs,LIL BRINCE - DEATH WISH,https://i.ytimg.com/vi/87aJyyoAREs/mqdefault.jpg,SCROLL MERCH AVAILABLE CHECK: https://scroll-gang.com/merch SCROLL official Spotify: https://sptfy.com/9gWy Artist ...,2019-06-01T19:15:02Z,1040000,81990,74.00,198.66,154.17,104.38
4915,UCJOi2N30HFXM4_rbwF_J2pQ,MIKA LASH MUSIQUE,iXBf921CCBs,Mika Lash - Je t&#39;emmène avec moi,https://i.ytimg.com/vi/iXBf921CCBs/mqdefault.jpg,"1er extrait de la mixtape ""Artiste"" disponible sur spotify : https://open.spotify.com/album/7aafaHQgZZK5Si0i4DCJtS Disponible ...",2019-06-01T15:31:41Z,59700,136581,123.27,137.86,51.27,150.28
4916,UCuvdmRlSWAlwB_rJZgg1cmQ,HYUKOH,625-jjwRdfw,[Official Audio] HYUKOH(혁오) - Tokyo Inn,https://i.ytimg.com/vi/625-jjwRdfw/mqdefault.jpg,[Official Audio] HYUKOH(혁오) - Tokyo Inn Artist : HYUKOH(혁오) Album Title : 23 Release : 2017.04.24 HYUKOH Official - Apple ...,2019-06-01T08:00:02Z,279000,195515,176.46,84.64,61.7,127.64
4917,UCMQ0kUQ13dTJi3026OF8sFA,Daily Busking,3pHeiWQezg4,[몰래카메라] 버스킹중 실제가수 난입.. 강제로 고음 헬파티함 ㅋㅋ (박혜원 - 시든꽃에물을주듯),https://i.ytimg.com/vi/3pHeiWQezg4/mqdefault.jpg,최근 신곡이 역주행하며 인지도 떡상중인 HYNN(박혜원)이 #시든꽃에물을주듯 커버 콘테스트 우승자 탈급식 조윤주양의 버스킹에 ...,2019-06-01T04:48:49Z,1610000,15105571,13633.19,132.04,47.53,148.61


In [ ]:
# 대표 컬러 시각화를 위한 R,G,B 값 정렬
# Random forest 모델 돌릴 때는 H,S,V 값을 추출했지만 시각화 용으로는 R,G,B 값을 추출함  
# 그래서 시각화 코드에는 R,G,B 기준으로 되어있음
df_music_visual = df_music.sort_values(by=['R','G','B'])
df_music_visual = df_music_visual.reset_index()
df_animal_visual = df_animal.sort_values(by=['R','G','B'])
df_animal_visual = df_animal_visual.reset_index()

In [ ]:
#r 값이 없는 행은 삭제 (떨구기..)
df_music_visual.drop(df_music_visual.loc[df_music_visual['R']==' '].index, inplace=True)
df_animal_visual.drop(df_animal_visual.loc[df_animal_visual['R']==' '].index, inplace=True)

In [ ]:
df_music_visual

In [ ]:
len(df_animal_visual)

6421

In [ ]:
#invalid literal for int() with base 10: ''
#위 오류가 발생한 원인은 4063번째부터 r,g,b 값이 없었기 때문임. 그래서 해당 행을 다 날려줌.. 위에서 
line=0
first_x=0
first_y=0
final_x=15
final_y=15

img_music = np.zeros((1615, 1615, 3), dtype = "uint8")

for i in range(0,len(df_music_visual)):
  try:
   a=int(df_music_visual['R'][i])
   b=int(df_music_visual['G'][i])
   c=int(df_music_visual['B'][i])
   img_music=cv2.rectangle(img_music,(first_x,first_y), (final_x,final_y),(a,b,c),-1)
   first_x=first_x+20
   final_x=final_x+20
   line=line+1
   if line%81==0:
     first_y=first_y+20
     final_y=final_y+20
     first_x=0
     final_x=15
  except Exception:
   pass    

In [ ]:
#invalid literal for int() with base 10: ''
#위 오류가 발생한 원인은 4063번째부터 r,g,b 값이 없었기 때문임. 그래서 해당 행을 다 날려줌.. 위에서 
line=0
first_x=0
first_y=0
final_x=15
final_y=15

img_animal = np.zeros((1615, 1615, 3), dtype = "uint8")

for i in range(0,len(df_animal_visual)):
  try:
   a=int(df_animal_visual['R'][i])
   b=int(df_animal_visual['G'][i])
   c=int(df_animal_visual['B'][i])
   img_animal=cv2.rectangle(img_animal,(first_x,first_y), (final_x,final_y),(a,b,c),-1)
   first_x=first_x+20
   final_x=final_x+20
   line=line+1
   if line%81==0:
     first_y=first_y+20
     final_y=final_y+20
     first_x=0
     final_x=15

  except Exception:
   pass  

In [ ]:
cv2_imshow(img_music)

In [ ]:
cv2_imshow(img_animal)